Import packages

In [1]:
# Everyone participated
import pandas as pd
import psycopg2
from sqlalchemy import create_engine
import matplotlib
matplotlib.use('TkAgg')  
import matplotlib.pyplot as plt
import easygui as eg
import numpy as np
from ipywidgets import interact
from scipy import stats
from scipy.stats import f_oneway
import matplotlib.ticker as mticker
import statsmodels.api as sm
from statsmodels.formula.api import ols

Load brightspace files

In [2]:
# Everyone participated
df1 = pd.read_excel('metaClean43Brightspace.xlsx')
df2 = pd.read_excel('sales.xlsx')
df3 = pd.read_excel('ExpertReviewsClean43LIWC.xlsx')
df4 = pd.read_excel('UserReviewsClean43LIWC.xlsx')

Copy dataframes

In [3]:
# Everyone participated, this is done to avoid going to the file loading process over and over agian while adjusting the code
movies_df = df1.copy()
sales_df = df2.copy()       
cr_df = df3.copy()
ur_df = df4.copy()

Movies file data cleaning

In [4]:
# This is done by Jochem Vis

# Drop not used columns
movies_df.drop(columns=['studio', 'rating', 'runtime', 'cast', 'director', 'summary', 'awards'])

# Add a movie_id column
movies_df['movie_id'] = range(1, len(movies_df) + 1)

# Rearrange the columns 
movies_df = movies_df[['movie_id', 'title', 'RelDate', 'genre', 'metascore', 'userscore', 'url']]

# Drop duplicates
movies_df = movies_df.drop_duplicates(subset='title')


Sales file cleaning

In [5]:
# This is done by Jayshree Sharma

# Drop not used columns
sales_df = sales_df.drop(columns=["year", "release_date", "international_box_office", "domestic_box_office", "Unnamed: 8", "avg run per theatre", "runtime", "keywords", "creative_type"])

# Add a sales_id column
sales_df["sales_id"] = range(1, len(sales_df) +1)

# Rearange the columns
sales_df = sales_df[['sales_id', 'title', 'genre', 'url', 'worldwide_box_office', 'production_budget', 'opening_weekend', 'theatre_count']]

# Drop duplicates
sales_df = sales_df.drop_duplicates(subset='title')

User review file cleaning

In [6]:
# This is done by Daan van der Veldt

# Drop duplicates
ur_df.drop_duplicates(subset='Rev', inplace=True)

# Drop not used columns
ur_df = ur_df[['url', 'idvscore', 'dateP']]

# Add a us_id column
ur_df['ur_id'] = range(1, len(ur_df) + 1)

# Rearrange the columns
ur_df = ur_df[['ur_id', 'url', 'idvscore', 'dateP']]

Critical review file cleaning

In [7]:
# This is done by Jelle Schelvis

# Drop duplicates
cr_df.drop_duplicates(subset='Rev', inplace=True)

# Drop not used columns
cr_df = cr_df[['url', 'idvscore', 'dateP']]

# Add a us_id column
cr_df['cr_id'] = range(1, len(cr_df) + 1)

# Rearrange the columns
cr_df = cr_df[['cr_id', 'url', 'idvscore', 'dateP']]

Create dataframe of unique genres

In [8]:
# This by done by Daan van der Veldt

# Merge the DataFrames including genre columns
genre_df = movies_df.merge(sales_df, on='title', how='inner')

# Drop not used columns
genre_df = genre_df[['title', 'genre_x', 'genre_y']]

# Split the 'genres' column by comma and create new columns for each genre
split_genres = genre_df['genre_x'].str.split(',', expand=True)
genre_df.loc[:, 'genre2'] = split_genres[0]
genre_df.loc[:, 'genre3'] = split_genres[1]
genre_df.loc[:, 'genre4'] = split_genres[2]
genre_df.loc[:, 'genre5'] = split_genres[3]
genre_df.loc[:, 'genre6'] = split_genres[4]
genre_df.loc[:, 'genre7'] = split_genres[5]
genre_df.loc[:, 'genre8'] = split_genres[6]
genre_df.loc[:, 'genre9'] = split_genres[7]
genre_df.loc[:, 'genre10'] = split_genres[8]
genre_df.loc[:, 'genre11'] = split_genres[9]

# Drop the split genre column and rename the first genre column
genre_df = genre_df.drop(columns=['genre_x'])
genre_df = genre_df.rename(columns={'genre_y': 'genre1'})

# Check if the merged columns do not contain duplicate genre, if so delete the second occurance
columns_to_check = ['genre2', 'genre3', 'genre4', 'genre5', 'genre6', 'genre7', 'genre8', 'genre9', 'genre10']
for index, row in genre_df.iterrows():
    genre1_value = row['genre1']
    for col in columns_to_check:
        if row[col] == genre1_value:
            genre_df.at[index, col] = None 

# Concatenate the 'genre' columns into a single Series
genre_series = genre_df[['genre1', 'genre2', 'genre3', 'genre4', 'genre5', 'genre6', 'genre7', 'genre8', 'genre9', 'genre10', 'genre11']].stack().reset_index(drop=True)

# Get unique genre values and assign IDs
unique_genres = genre_series.unique()
genre_to_id = {genre: id for id, genre in enumerate(unique_genres)}

# Create a DataFrame with genre IDs
unique_genres_df = pd.DataFrame({'genre': unique_genres, 'genre_id': [genre_to_id[genre] for genre in unique_genres]})

# Add a genre_id column
unique_genres_df['genre_id'] = unique_genres_df['genre_id'] + 1

# Rearrange the columns
unique_genres_df = unique_genres_df[['genre_id', 'genre']]


Create datafame of genres per movie

In [9]:
# This is done by Daan van der Veldt

# Merge the DataFrames including genre columns
genre_rel_df = movies_df.merge(sales_df, on='title', how='inner')

# Drop not used columns
genre_rel_df = genre_rel_df[['title', 'genre_x', 'genre_y']]

# Split the 'genres' column by comma and create new columns for each genre
split_genres = genre_rel_df['genre_x'].str.split(',', expand=True)
genre_rel_df.loc[:, 'genre2'] = split_genres[0]
genre_rel_df.loc[:, 'genre3'] = split_genres[1]
genre_rel_df.loc[:, 'genre4'] = split_genres[2]
genre_rel_df.loc[:, 'genre5'] = split_genres[3]
genre_rel_df.loc[:, 'genre6'] = split_genres[4]
genre_rel_df.loc[:, 'genre7'] = split_genres[5]
genre_rel_df.loc[:, 'genre8'] = split_genres[6]
genre_rel_df.loc[:, 'genre9'] = split_genres[7]
genre_rel_df.loc[:, 'genre10'] = split_genres[8]
genre_rel_df.loc[:, 'genre11'] = split_genres[9]

# Drop the split genre column and rename the first genre column
genre_rel_df = genre_rel_df.drop(columns=['genre_x'])
genre_rel_df = genre_rel_df.rename(columns={'genre_y': 'genre1'})

# Check if the merged columns do not contain duplicate genre, if so delete the second occurance
columns_to_check = ['genre2', 'genre3', 'genre4', 'genre5', 'genre6', 'genre7', 'genre8', 'genre9', 'genre10']
for index, row in genre_rel_df.iterrows():
    genre1_value = row['genre1']
    for col in columns_to_check:
        if row[col] == genre1_value:
            genre_rel_df.at[index, col] = None 

# Concatenate the 'genre' columns into a single Series
genre_series = genre_rel_df[['genre1', 'genre2', 'genre3', 'genre4', 'genre5', 'genre6', 'genre7', 'genre8', 'genre9', 'genre10', 'genre11']].stack().reset_index(drop=True)

# Create a list of genres
genre_columns = [f'genre{i}' for i in range(1, 12)]

# Create a new column 'Combined_Genres' by combining genre columns into lists
genre_rel_df['Combined_Genres'] = genre_rel_df.apply(lambda row: [row[col] for col in [f'genre{i}' for i in range(1, 12)] if pd.notna(row[col])], axis=1)

# Drop the individual genre columns
genre_rel_df.drop(columns=genre_columns, inplace=True)

# Reset the index
genre_rel_df.reset_index(drop=True, inplace=True)

# Explode the 'Combined_Genres' column to create separate rows
genre_rel_df = genre_rel_df.explode('Combined_Genres', ignore_index=True)

# Reset the index
genre_rel_df.reset_index(drop=True, inplace=True)

# Rename column for merging
genre_rel_df = genre_rel_df.rename(columns={'Combined_Genres': 'genre'})

# Perform a left merge
genre_rel_df = genre_rel_df.merge(unique_genres_df, on='genre', how='left')

# Add a genre_rel_id column
genre_rel_df['genre_rel_id'] = range(1, len(genre_rel_df) + 1)

# Rearrange the columns
genre_rel_df = genre_rel_df[['genre_rel_id', 'genre_id', 'title']]

Merge foreign keys to dataframes

In [10]:
# This is done by everyone, because everyone merged thei foreign key to the database they were responsible for

# Merge foreign keys
movies_df = movies_df.merge(sales_df[['title', 'sales_id']], on='title', how='left')
sales_df = sales_df.merge(movies_df[['title', 'movie_id']], on='title', how='left')
ur_df = ur_df.merge(movies_df[['url', 'movie_id']], on='url', how='left')
cr_df = cr_df.merge(movies_df[['url', 'movie_id']], on='url', how='left')

# Delete no longer needed column
sales_df.drop(columns=['genre'], inplace=True)
movies_df.drop(columns=['genre'], inplace=True)

# Merge foerign key to genre relations and set up the data
genre_rel_df = genre_rel_df.merge(movies_df[['title', 'movie_id']], on='title', how='left')
genre_rel_df.drop(columns='title', inplace=True)
genre_rel_df = genre_rel_df[['genre_rel_id', 'genre_id', 'movie_id']]

# Drop some final unneeded columns
movies_df.drop(columns='url', inplace=True)
sales_df.drop(columns='title', inplace=True)
sales_df.drop(columns='url', inplace=True)
sales_df.drop(columns='opening_weekend', inplace=True)
sales_df.drop(columns='theatre_count', inplace=True)
ur_df.drop(columns='url', inplace=True)
ur_df.drop(columns='dateP', inplace=True)
cr_df.drop(columns='url', inplace=True)
cr_df.drop(columns='dateP', inplace=True)


Make database connection and delete all tables and create a SQLAlchemy engine

In [11]:
# This is done by Daan van der Veldt

# Create a function to show the input dialog
def get_db_connection_details():
    msg = "Enter Database Connection Details"
    title = "Database Connection"
    field_names = ["Host", "Database Name", "Username", "Password"]
    default_values = ["localhost", "postgres", "myuser", "mypassword"]

    field_values = eg.multenterbox(msg, title, field_names, default_values)

    # Check if the user canceled the input dialog
    if field_values is None:
        return None

    return {
        "host": field_values[0],
        "database": field_values[1],
        "user": field_values[2],
        "password": field_values[3]
    }

# Get database connection details from the user
db_details = get_db_connection_details()

if db_details is not None:
    # Create a new connection
    connection = psycopg2.connect(
        host=db_details["host"],
        database=db_details["database"],
        user=db_details["user"],
        password=db_details["password"]
    )

# Create cursor
cursor = connection.cursor()

# Get a list of all table names in the current schema
cursor.execute("SELECT table_name FROM information_schema.tables WHERE table_schema = 'public'")

table_names = [table[0] for table in cursor.fetchall()]

for table_name in table_names:
    cursor.execute(f"DROP TABLE IF EXISTS {table_name} CASCADE")

connection.commit()

# Create a SQLAlchemy engine using the obtained connection details
engine = create_engine(f"postgresql://{db_details['user']}:{db_details['password']}@{db_details['host']}/{db_details['database']}")


Import tables into SQL database

In [12]:
# This is done by Daan van der Veldt

# Import the movies data into SQL
movies_df.to_sql('movies', engine, if_exists='replace', index=False)

# Import the sales data into SQL
sales_df.to_sql('sales', engine, if_exists='replace', index=False)

# Import the user review data into SQL
ur_df.to_sql('user_reviews', engine, if_exists='replace', index=False)

# Import the critical review data into SQL
cr_df.to_sql('critical_reviews', engine, if_exists='replace', index=False)

# Import the unique genres data into SQL
unique_genres_df.to_sql('unique_genres', engine, if_exists='replace', index=False)

# Import the genres relations data into SQL
genre_rel_df.to_sql('genre_rel', engine, if_exists='replace', index=False)

491

Set primary keys and foreign keys

In [13]:
# This is done by Daan van der Veldt

# Create a cursor object
cursor = connection.cursor()

sql_command = """
    ALTER TABLE movies 
    ADD CONSTRAINT movie_id PRIMARY KEY (movie_id);

    ALTER TABLE sales 
    ADD CONSTRAINT sales_id PRIMARY KEY (sales_id);

    ALTER TABLE user_reviews 
    ADD CONSTRAINT ur_id PRIMARY KEY (ur_id);

    ALTER TABLE critical_reviews 
    ADD CONSTRAINT cr_id PRIMARY KEY (cr_id);

    ALTER TABLE genre_rel 
    ADD CONSTRAINT genre_rel_id PRIMARY KEY (genre_rel_id);

    ALTER TABLE unique_genres 
    ADD CONSTRAINT genre_id PRIMARY KEY (genre_id);

    ALTER TABLE movies
    ALTER COLUMN sales_id TYPE BIGINT;

    ALTER TABLE sales
    ALTER COLUMN movie_id TYPE BIGINT;

    ALTER TABLE genre_rel
    ALTER COLUMN genre_id TYPE BIGINT;

    ALTER TABLE critical_reviews
    ALTER COLUMN movie_id TYPE BIGINT;

    ALTER TABLE user_reviews
    ALTER COLUMN movie_id TYPE BIGINT;

    ALTER TABLE movies 
    ADD CONSTRAINT fk_sales_id
    FOREIGN KEY (sales_id) REFERENCES sales (sales_id);

    ALTER TABLE sales 
    ADD CONSTRAINT fk_movie_id
    FOREIGN KEY (movie_id) REFERENCES movies (movie_id);

    ALTER TABLE user_reviews 
    ADD CONSTRAINT fk_movie_id
    FOREIGN KEY (movie_id) REFERENCES movies (movie_id);

    ALTER TABLE critical_reviews
    ADD CONSTRAINT fk_movie_id
    FOREIGN KEY (movie_id) REFERENCES movies (movie_id);

    ALTER TABLE genre_rel
    ADD CONSTRAINT fk_genre_id
    FOREIGN KEY (genre_id) REFERENCES unique_genres (genre_id);

    ALTER TABLE genre_rel
    ADD CONSTRAINT fk_movie_id
    FOREIGN KEY (movie_id) REFERENCES movies (movie_id);
"""
# Execute the SQL command
cursor.execute(sql_command)

# Commit the changes to the database
connection.commit()


Change datatypes to match ERD

In [14]:
# This is done by Jelle Schelvis

cursor = connection.cursor()
sql_command = """
    ALTER TABLE critical_reviews
    ALTER COLUMN idvscore
    SET DATA TYPE INT;

    ALTER TABLE movies
    ALTER COLUMN title 
    SET DATA TYPE VARCHAR(200);

    ALTER TABLE movies
    ALTER COLUMN metascore 
    SET DATA TYPE INT;

    ALTER TABLE movies
    ALTER COLUMN "RelDate"
    SET DATA TYPE DATE;

    ALTER TABLE movies
    ALTER COLUMN metascore 
    SET DATA TYPE int;

    ALTER TABLE movies
    ALTER COLUMN userscore 
    SET DATA TYPE int;

    ALTER TABLE unique_genres
    ALTER COLUMN genre 
    SET DATA TYPE VARCHAR(30);

    ALTER TABLE sales
    ALTER COLUMN worldwide_box_office
    SET DATA TYPE BIGINT;

    ALTER TABLE sales
    ALTER COLUMN production_budget
    SET DATA TYPE INT;   

    ALTER TABLE user_reviews
    ALTER COLUMN idvscore
    SET DATA TYPE int;
"""

# Execute the SQL command
cursor.execute(sql_command)

# Commit the changes to the database
connection.commit()

Create plots for hypothesises

In [15]:
# This is done by everyone, Jelle Schelvis created hypothesis 1 and 2, Daan van der Veldt created hypothesis 3 and 4, Jayshree Sharmma created hhypothesis 5 and 6, and Jochem Vis created hypothesis 7 and 8. The display screen coding was done by Daan van der Veldt.

# Function to display a hypothesis plot based on the hypothesis number
def display_hypothesis_plot(hypothesis_number):


    if hypothesis_number == 1:

        title = f'Hypothesis {hypothesis_number}: Expert reviews impact box office'
        sql_command = """
        SELECT cr.movie_id, 
        ROUND(AVG(cr.idvscore)::numeric, 2) AS avg_idvscore,
        ROUND(s.worldwide_box_office::numeric, 2) AS worldwide_box_office
        FROM critical_reviews cr
        INNER JOIN sales s ON s.movie_id = cr.movie_id
        WHERE s.worldwide_box_office IS NOT NULL
        AND cr.idvscore IS NOT NULL
        GROUP BY cr.movie_id, s.worldwide_box_office;
        """
        cursor.execute(sql_command)
        data = cursor.fetchall()
        gemiddelde_idvscore = [float(column[1]) for column in data]
        worldwide_box_office = [float(column[2]) for column in data]

        pearson_corr, p_value = stats.pearsonr(gemiddelde_idvscore, worldwide_box_office)

        slope, intercept = np.polyfit(gemiddelde_idvscore, worldwide_box_office, 1)
        trendline = slope * np.array(gemiddelde_idvscore) + intercept

        # Calculate residuals
        residuals = np.array(worldwide_box_office) - trendline

        # Calculate Z-scores for residuals
        z_scores = np.abs(stats.zscore(residuals))

        # Set a threshold for Z-scores to identify outliers
        z_threshold = 2.0

        # Filter data to exclude outliers
        filtered_gemiddelde_idvscore = []
        filtered_box_office = []
        for i, z_score in enumerate(z_scores):
            if z_score <= z_threshold:
                filtered_gemiddelde_idvscore.append(gemiddelde_idvscore[i])
                filtered_box_office.append(worldwide_box_office[i])

        plt.ion()
        fig, ax = plt.subplots(figsize=(8, 6))
        ax.scatter(filtered_gemiddelde_idvscore, filtered_box_office, label=f'Hypothesis {hypothesis_number}', alpha=0.6, s=10)
        ax.plot(gemiddelde_idvscore, trendline, color='red', label='Trendline')
        ax.set_title(title)
        ax.set_xlabel(f'Expert Scores \n(Pearson Correlation Coefficient: {pearson_corr:.4f}, Significance: {p_value:.4f})')        
        ax.set_ylabel('Worldwide Box Office')
        ax.yaxis.set_major_formatter(mticker.FuncFormatter(lambda x, pos: f'{x / 1e6:.0f}M'))
        ax.legend()
        ax.grid(True)
        plt.show(block=False)

        # pearson correlation analysis
        print(f"Pearson Correlation Coefficient: {pearson_corr:.4f}")
        print(f"P-value: {p_value:.4f}")

    elif hypothesis_number == 2:
        title = f'Hypothesis {hypothesis_number}: Expert reviews impact box office category drama'

        sql_command = """
        SELECT cr.movie_id, 
        ROUND(AVG(cr.idvscore)::numeric, 2) AS avg_idvscore,
        ROUND(s.worldwide_box_office::numeric, 2) AS worldwide_box_office
        FROM critical_reviews cr
        INNER JOIN sales s ON s.movie_id = cr.movie_id AND s.worldwide_box_office IS NOT NULL
        INNER JOIN genre_rel gr ON gr.movie_id = cr.movie_id
        INNER JOIN unique_genres ug ON ug.genre_id = gr.genre_id
        GROUP BY cr.movie_id, s.worldwide_box_office, ug.genre
        HAVING AVG(cr.idvscore) IS NOT NULL
        AND ug.genre = 'Drama';
        ;
        """
        cursor.execute(sql_command)
        data = cursor.fetchall()
        gemiddelde_idvscore = [float(column[1]) for column in data]
        worldwide_box_office = [float(column[2]) for column in data]

        pearson_corr, p_value = stats.pearsonr(gemiddelde_idvscore, worldwide_box_office)

        slope, intercept = np.polyfit(gemiddelde_idvscore, worldwide_box_office, 1)
        trendline = slope * np.array(gemiddelde_idvscore) + intercept

        # Calculate residuals
        residuals = np.array(worldwide_box_office) - trendline

        # Calculate Z-scores for residuals
        z_scores = np.abs(stats.zscore(residuals))

        # Set a threshold for Z-scores to identify outliers
        z_threshold = 2.0 

        # Filter data to exclude outliers
        filtered_gemiddelde_idvscore = []
        filtered_box_office = []
        for i, z_score in enumerate(z_scores):
            if z_score <= z_threshold:
                filtered_gemiddelde_idvscore.append(gemiddelde_idvscore[i])
                filtered_box_office.append(worldwide_box_office[i])

        plt.ion()
        fig, ax = plt.subplots(figsize=(8, 6))
        ax.scatter(filtered_gemiddelde_idvscore, filtered_box_office, label=f'Hypothesis {hypothesis_number}', alpha=0.6, s=10)
        ax.plot(gemiddelde_idvscore, trendline, color='red', label='Trendline')
        ax.set_title(title)
        ax.set_xlabel(f'Expert Scores \n(Pearson Correlation Coefficient: {pearson_corr:.4f}, Significance: {p_value:.4f})')        
        ax.set_ylabel('Worldwide Box Office')
        ax.yaxis.set_major_formatter(mticker.FuncFormatter(lambda x, pos: f'{x / 1e6:.0f}M'))
        ax.legend()
        ax.grid(True)
        plt.show(block=False)


    elif hypothesis_number == 3:
        title = f'Hypothesis {hypothesis_number}: Negative user scores and box office'
        
        # Select SQL query and use data
        sql_command = """
            SELECT ur.movie_id, AVG(ur.idvscore) AS avg_idvscore, s.worldwide_box_office
            FROM user_reviews ur
            INNER JOIN sales s ON s.movie_id = ur.movie_id
            WHERE s.worldwide_box_office IS NOT NULL
            GROUP BY ur.movie_id, s.worldwide_box_office;
        """
        cursor.execute(sql_command)
        data = cursor.fetchall()
        userscores = [float(column[1]) for column in data]
        worldwide_box_office = [float(column[2]) for column in data]
        slope, intercept = np.polyfit(userscores, worldwide_box_office, 1)
        trendline = slope * np.array(userscores) + intercept

        pearson_corr, p_value = stats.pearsonr(userscores, worldwide_box_office)

        # Calculate residuals
        residuals = np.array(worldwide_box_office) - trendline

        # Calculate Z-scores for residuals
        z_scores = np.abs(stats.zscore(residuals))

        # Set a threshold for Z-scores to identify outliers
        z_threshold = 2.0 

        # Filter data to exclude outliers
        filtered_userscores = []
        filtered_box_office = []
        for i, z_score in enumerate(z_scores):
            if z_score <= z_threshold:
                filtered_userscores.append(userscores[i])
                filtered_box_office.append(worldwide_box_office[i])

        # Create a plot
        plt.ion()
        fig, ax = plt.subplots(figsize=(8, 6))

        # Make the main plot
        ax.scatter(filtered_userscores, filtered_box_office, label=f'Hypothesis {hypothesis_number}', alpha=0.6, s=5)
        ax.plot(userscores, trendline, color='red', label='Trendline')
        ax.set_title(title)
        ax.set_xlabel(f'User Scores \n (Pearson Correlation Coefficient: {pearson_corr:.4f}, Significance: {p_value:.4f})')
        ax.set_ylabel('Worldwide Box Office')
        ax.legend()
        ax.yaxis.set_major_formatter(mticker.FuncFormatter(lambda x, pos: f'{x / 1e6:.0f}M'))
        ax.grid(True)
        plt.show(block=False)


    elif hypothesis_number == 4:
        title = f'Hypothesis {hypothesis_number}: User reviews and box office'

        # Select SQL query and use that data
        sql_command = """
            SELECT ur.movie_id,
            COUNT(ur.ur_id) AS user_review_count,
            s.worldwide_box_office
            FROM user_reviews ur
            INNER JOIN sales s ON s.movie_id = ur.movie_id
            WHERE s.worldwide_box_office IS NOT NULL
            GROUP BY ur.movie_id, s.worldwide_box_office;
        """
        cursor.execute(sql_command)
        data = cursor.fetchall()
        user_review_count = [float(column[1]) for column in data]
        worldwide_box_office = [float(column[2]) for column in data]

        pearson_corr, p_value = stats.pearsonr(user_review_count, worldwide_box_office)

        # Create trendline
        slope, intercept = np.polyfit(user_review_count, worldwide_box_office, 1)
        trendline = slope * np.array(user_review_count) + intercept

        # Calculate residuals
        residuals = np.array(worldwide_box_office) - trendline

        # Calculate Z-scores for residuals
        z_scores = np.abs(stats.zscore(residuals))

        # Set a threshold for Z-scores to identify outliers
        z_threshold = 2.0  

        # Filter data to exclude outliers
        filtered_user_review_count = []
        filtered_box_office = []
        for i, z_score in enumerate(z_scores):
            if z_score <= z_threshold:
                filtered_user_review_count.append(user_review_count[i])
                filtered_box_office.append(worldwide_box_office[i])

        y_min = min(filtered_box_office)
        y_max = max(filtered_box_office)


        # Create the main plot and the zoomed-in subplot within the same figure
        fig, ax = plt.subplots(1, 2, figsize=(12, 6)) 

        # Make the main plot
        ax[0].scatter(filtered_user_review_count, filtered_box_office, label='Main Plot', alpha=0.6, s=10)
        ax[0].plot(user_review_count, trendline, color='red', label='Trendline')
        ax[0].set_title(title)
        ax[0].set_xlabel(f'User Score Count per movie \n (Pearson Correlation Coefficient: {pearson_corr:.4f}, Significance: {p_value:.4f})')
        ax[0].set_ylabel('Worldwide Box Office (in millions)')  
        ax[0].set_xlim(0, 1000)
        ax[0].set_ylim(y_min, y_max)  

        # Format the y-axis tick labels to display values in millions
        ax[0].yaxis.set_major_formatter(mticker.FuncFormatter(lambda x, pos: f'{x / 1e6:.0f}M'))
        ax[0].legend()
        ax[0].grid(True)

        # Create the zoomed-in subplot
        ax[1].scatter(filtered_user_review_count, filtered_box_office, alpha=0.6, s=1)
        ax[1].plot(user_review_count, trendline, color='red')
        ax[1].set_title('Zoomed in fist grid')
        ax[1].set_xlabel('User Score Count per movie')
        ax[1].set_ylabel('Worldwide Box Office (in millions)') 
        ax[1].set_xlim(0, 200)  
        ax[1].set_ylim(0, 200000000)  

        # Format the y-axis tick labels to display values in millions
        ax[1].yaxis.set_major_formatter(mticker.FuncFormatter(lambda x, pos: f'{x / 1e6:.0f}M'))
        ax[1].grid(True)
        plt.tight_layout()
        plt.show()


    elif hypothesis_number == 5:
        title = f'Hypothesis {hypothesis_number}: Movie release in holdiday season'

        # Select SQL query and use that data for the ANOVA test
        sql_command = """
        SELECT
        (CASE WHEN EXTRACT(MONTH FROM "RelDate") = 12 THEN "worldwide_box_office" ELSE NULL END) AS average_worldwide_box_office_christmas,
        ("worldwide_box_office") AS average_worldwide_box_office,
        (CASE WHEN EXTRACT(MONTH FROM "RelDate") BETWEEN 6 AND 9  THEN "worldwide_box_office" ELSE NULL END) AS average_worldwide_box_office_summer,
        (CASE WHEN EXTRACT(MONTH FROM "RelDate") IN (1,2,3,4,5,10,11)  THEN "worldwide_box_office" ELSE NULL END) AS average_worldwide_box_office_outsideholiday
        FROM sales
            INNER JOIN movies ON movies.movie_ID = sales.movie_ID;
        """
        cursor.execute(sql_command)
        data = cursor.fetchall()
 
        # Extract the required columns for the ANOVA test
        christmas_data = [column[1] for column in data if column[1] is not None]
        summer_data = [column[2] for column in data if column[2] is not None]
        overall_data = [column[0] for column in data if column[0] is not None]
        outside_holiday_data = [column[3] for column in data if column[3] is not None]

        # Perform the ANOVA test
        f_statistic, p_value = f_oneway(christmas_data, summer_data, overall_data, outside_holiday_data)

        # Output the results
        print(f"F-statistic: {f_statistic:.4f}")
        print(f"P-value: {p_value:.4f}")

        # Select SQL query and use that data for the chart
        sql_command = """ 
            SELECT
            AVG(CASE WHEN EXTRACT(MONTH FROM "RelDate") = 12 THEN "worldwide_box_office" ELSE NULL END) AS average_worldwide_box_office_christmas,
            AVG("worldwide_box_office") AS average_worldwide_box_office,
            AVG(CASE WHEN EXTRACT(MONTH FROM "RelDate") BETWEEN 6 AND 9  THEN "worldwide_box_office" ELSE NULL END) AS average_worldwide_box_office_summer,
            AVG(CASE WHEN EXTRACT(MONTH FROM "RelDate") IN (1,2,3,4,5,10,11)  THEN "worldwide_box_office" ELSE NULL END) AS average_worldwide_box_office_outsideholiday
            FROM sales
                INNER JOIN movies ON movies.movie_ID = sales.movie_ID
         """
        cursor.execute(sql_command)
        data = cursor.fetchall()
 
        # Create variable season
        season = ['overall', 'christmas', 'summer']
        
        # Extract individual float values from the data
        box_office_overall = [float(column[0]) for column in data]
        box_office_christmas = [float(column[1]) for column in data]
        box_office_summer = [float(column[2]) for column in data]
                
        x_positions = range(len(season))
    
        # Create bar charts for each season
        fig, ax = plt.subplots(figsize=(8, 6))
        ax.bar(season[1], box_office_christmas, color='magenta', label='Christmas')
        ax.bar(season[0], box_office_overall, color='lightgreen', label='Overall')
        ax.bar(season[2], box_office_summer, color='cyan', label='Summer')
                
        ax.set_xticks(x_positions, season)
        ax.set_xlabel(f'Release Season \n ("F-statistic: {f_statistic:.4f}", "P-value: {p_value:.4f}" )')
        ax.set_ylabel('Box_Office')
        ax.set_title('Box_office per Season')
        ax.grid(True)
        ax.yaxis.set_major_formatter(mticker.FuncFormatter(lambda x, pos: f'{x / 1e6:.0f}M'))
        plt.show()

    elif hypothesis_number == 6:
        title = f'Hypothesis {hypothesis_number}: Movie release outside holiday season'

        # Select SQL query and use that data for the chart 
        sql_command = """ 
            SELECT
 	        AVG(CASE WHEN EXTRACT(MONTH FROM "RelDate") = 12 THEN "worldwide_box_office" ELSE NULL END) AS average_worldwide_box_office_christmas,
  	        AVG("worldwide_box_office") AS average_worldwide_box_office,
  	        AVG(CASE WHEN EXTRACT(MONTH FROM "RelDate") BETWEEN 6 AND 9  THEN "worldwide_box_office" ELSE NULL END) AS average_worldwide_box_office_summer,
	        AVG(CASE WHEN EXTRACT(MONTH FROM "RelDate") IN (1,2,3,4,5,10,11)  THEN "worldwide_box_office" ELSE NULL END) AS average_worldwide_box_office_outsideholiday
            FROM sales
            INNER JOIN movies ON movies.movie_ID = sales.movie_ID;
         """
        cursor.execute(sql_command)
        data = cursor.fetchall()
        
        # Create variable season
        season = ['overall', 'outside_holiday']
      
        # Extract individual float values from the data
        box_office_overall = [float(column[1]) for column in data]
        box_office_outsideholiday = [float(column[3]) for column in data]

        x_positions = range(len(season))

        # Create bar charts for each season
        fig, ax = plt.subplots(figsize=(8, 6))
        ax.bar(season[0], box_office_overall, color='blue', label='Overall')
        ax.bar(season[1], box_office_outsideholiday, color='cyan', label='Outside Holiday')

        ax.set_xticks(x_positions, season)
        ax.set_xlabel('Release Season')
        ax.set_ylabel('Box_Office')
        ax.set_title('Box_office per Season')
        ax.grid(True)
        ax.yaxis.set_major_formatter(mticker.FuncFormatter(lambda x, pos: f'{x / 1e6:.0f}M'))
        plt.show()

    elif hypothesis_number == 7:
        title = f'Hypothesis {hypothesis_number}: Movie budget and box office'

        # Select SQL query and use that data for the ANOVA test
        sql_command = """
            SELECT
                CASE
                    WHEN production_budget IS NULL THEN 'Unknown'
                    WHEN production_budget < 1000000 THEN 'Very Low Budget'
                    WHEN production_budget >= 1000000 AND production_budget < 5000000 THEN 'Low Budget'
                    WHEN production_budget >= 5000000 AND production_budget < 10000000 THEN 'Medium-Low Budget'
                    WHEN production_budget >= 10000000 AND production_budget < 30000000 THEN 'Medium Budget'
                    WHEN production_budget >= 30000000 AND production_budget < 70000000 THEN 'Medium-High Budget'
                    WHEN production_budget >= 70000000 AND production_budget < 100000000 THEN 'High-Medium Budget'
                    WHEN production_budget >= 100000000 THEN 'High Budget'
                END AS Budget_Category,
                worldwide_box_office AS Average_Box_Office_Sales
            FROM
                sales
            WHERE
                production_budget IS NOT NULL
        """

        # Execute the modified SQL query to retrieve data
        cursor.execute(sql_command)
        data = cursor.fetchall()

        # Create a DataFrame with the retrieved data
        sales_df = pd.DataFrame(data, columns=['budget_category', 'average_box_office_Sales'])

        # Fit the ANOVA model
        model = ols('average_box_office_Sales ~ C(budget_category)', data=sales_df).fit()

        # Perform ANOVA
        anova_table = sm.stats.anova_lm(model, typ=2)

       # Select the F-statistic and the p-value from the ANOVA table
        f_statistic = anova_table.loc['C(budget_category)', 'F']
        p_value = anova_table.loc['C(budget_category)', 'PR(>F)']

        # Print the F-statistic and the p-value
        print(f"F-statistic : {f_statistic}")
        print(f"P-value : {p_value}")

        # Select SQL query and use that data
        sql_command = """
            SELECT
                CASE
                    WHEN production_budget IS NULL THEN 'Unknown'
                    WHEN production_budget < 1000000 THEN 'Very Low Budget'
                    WHEN production_budget >= 1000000 AND production_budget < 5000000 THEN 'Low Budget'
                    WHEN production_budget >= 5000000 AND production_budget < 10000000 THEN 'Medium-Low Budget'
                    WHEN production_budget >= 10000000 AND production_budget < 30000000 THEN 'Medium Budget'
                    WHEN production_budget >= 30000000 AND production_budget < 70000000 THEN 'Medium-High Budget'
                    WHEN production_budget >= 70000000 AND production_budget < 100000000 THEN 'High-Medium Budget'
                    WHEN production_budget >= 100000000 THEN 'High Budget'
                END AS Budget_Category,
                AVG(worldwide_box_office) AS Average_Box_Office_Sales
            FROM
                sales
            WHERE
                production_budget IS NOT NULL
            GROUP BY
                Budget_Category
            ORDER BY
                Average_Box_Office_Sales DESC;
            """
        cursor.execute(sql_command)
        data = cursor.fetchall()
        budget_category = [(column[0]) for column in data]
        average_box_office_sales = [(column[1]) for column in data]

        # Make the plot
        fig, ax = plt.subplots(figsize=(13, 8))
        ax.bar(budget_category, average_box_office_sales, color='lightgreen')
        ax.set_xlabel('Budget Category')
        ax.set_ylabel('Average Box Office Sales (in millions)')
        ax.set_title('Average Box Office Sales per Budget Category')
        ax.yaxis.set_major_formatter(mticker.FuncFormatter(lambda x, pos: f'{x / 1e6:.0f}M'))
        ax.grid(True)
        
        # Add the F-value and p-value to the plot
        plt.text(0.95, -0.1, f'F-value: {f_statistic:.2f}', transform=ax.transAxes, fontsize=12, ha='right')
        plt.text(0.95, -0.15, f'P-value: {p_value:.4f}', transform=ax.transAxes, fontsize=12, ha='right')
        plt.tight_layout()
        plt.show(block=False)


    elif hypothesis_number == 8:
        title = f'Hypothesis {hypothesis_number}: Genres and box office sales'
        
        # Select SQL query and use that data
        sql_command = """ 
                SELECT g.Genre, AVG(s.worldwide_box_office) AS avg_sales
                FROM genre_rel AS gr
                JOIN sales AS s ON gr.movie_id = s.sales_id
                JOIN unique_genres AS g ON gr.genre_id = g.Genre_ID
                GROUP BY g.Genre
                ORDER BY avg_sales DESC;
            """
        cursor.execute(sql_command)
        data = cursor.fetchall()
        genre = [(colum[0]) for colum in data]
        worldwide_box_office = [(colum[1]) for colum in data]

        # Make the plot
        fig, ax = plt.subplots(figsize=(17, 8))
        ax.bar(genre, worldwide_box_office, color='skyblue')
        ax.set_xlabel('Genres')
        ax.set_ylabel('Box Office Sales (in millions)')
        ax.set_title('Box Office Sales per Genre')
        ax.set_xticks(range(len(genre)))
        ax.set_xticklabels(genre, rotation=45)   
        ax.yaxis.set_major_formatter(mticker.FuncFormatter(lambda x, pos: f'{x / 1e6:.0f}M'))
        ax.grid(True)
        plt.tight_layout()
        plt.show(block=False)


# Create a function to show the hypothesis selection dialog
def select_hypothesis_plot():
    while True:
        msg = "Select a Hypothesis Plot to Display"
        title = "Hypothesis Selection"
        choices = [
            "Hypothesis 1: Expert reviews impact box office",
            "Hypothesis 2: Expert reviews influence drama movies",
            "Hypothesis 3: Negative user scores and box office",
            "Hypothesis 4: User reviews and box office",
            "Hypothesis 5: Movie release timing",
            "Hypothesis 6: Movie release outside holiday season",
            "Hypothesis 7: Movie budget and box office",
            "Hypothesis 8: Genres and box office sales",
            "Exit"
        ]

        choice = eg.choicebox(msg, title, choices=choices)

        if choice is None or choice == "Exit":
            break

        # Extract the hypothesis number
        hypothesis_number = int(choice.split(":")[0].split()[-1])
        display_hypothesis_plot(hypothesis_number)

# Show the hypothesis selection dialog
select_hypothesis_plot()

F-statistic: 7.6382
P-value: 0.0000
